In [2]:
pip install typedb-client


     |████████████████████████████████| 41 kB 730 kB/s 
     |████████████████████████████████| 45 kB 6.4 MB/s 
     |████████████████████████████████| 3.9 MB 9.4 MB/s 
     |████████████████████████████████| 1.0 MB 49.7 MB/s 
  Created wheel for typedb-client: filename=typedb_client-2.4.0-py2.py3-none-any.whl size=103775 sha256=78a834523bd8e9260dea9fe8159b709d81fa6f37f5dfa7ab14309a62efe6bccc
  Stored in directory: /Users/joebrucker/Library/Caches/pip/wheels/ee/5b/88/ee6553c1327ab4545e8be068ac620947af6b8ba6472517d91c
  Created wheel for typedb-protocol: filename=typedb_protocol-2.4.0-py2.py3-none-any.whl size=59824 sha256=bc286d3f20068c9708e985c7d77ca87d2c12bdd4cb3d3025a0d3acfba8e91c39
  Stored in directory: /Users/joebrucker/Library/Caches/pip/wheels/48/b1/ca/3592a8e942d49f730e699ad60242af774c0ff4cfd2042199ed
Successfully built typedb-client typedb-protocol
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.18.0
    Uninstalling protobuf-3.18.0:
      Success

In [6]:
from typedb.client import *

In [14]:
with TypeDB.core_client("localhost:1729") as client:
    dbs = client.databases().all()

dbs

In [12]:
with TypeDB.core_client("localhost:1729") as client:
    client.databases().create("nlp twitter")

In [16]:
with TypeDB.core_client("localhost:1729") as client:
    client.users().create("admin")

AttributeError: '_CoreClient' object has no attribute 'users'

In [ ]:

with TypeDB.core_client("localhost:1729") as client:
    with client.session("social_network", SessionType.DATA) as session:
        ## Insert a Person using a WRITE transaction
        with session.transaction(TransactionType.WRITE) as write_transaction:
            insert_iterator = write_transaction.query().insert('insert $x isa person, has email "x@email.com";')
            concepts = [ans.get("x") for ans in insert_iterator]
            print("Inserted a person with ID: {0}".format(concepts[0].get_iid()))
            ## to persist changes, write transaction must always be committed (closed)
            write_transaction.commit()

        ## Read the person using a READ only transaction
        with session.transaction(TransactionType.READ) as read_transaction:
            answer_iterator = read_transaction.query().match("match $x isa person; get $x; limit 10;")

            for answer in answer_iterator:
                person = answer.get("x")
                print("Retrieved person with id " + person.get_iid())

        ## Or query and consume the iterator immediately collecting all the results
        with session.transaction(TransactionType.READ) as read_transaction:
            answer_iterator = read_transaction.query().match("match $x isa person; get $x; limit 10;")
            persons = [ans.get("x") for ans in answer_iterator]
            for person in persons:
                print("Retrieved person with id " + person.get_iid())

        ## if not using a `with` statement, then we must always close the session and the read transaction
        # read_transaction.close()
        # session.close()
        # client.close()

In [ ]:
define
  employment sub relation, relates employee, relates employer; 
  person sub entity, plays employment:employee;
  company sub entity, plays employment:employer;